In [1]:
import excited_workflow
from pathlib import Path
import xarray as xr
import xarray_regrid  # Importing this will make Dataset.regrid accessible.

In [2]:
from dask.distributed import Client
client = Client()

In [3]:
ds_cb = xr.open_dataset("/data/volume_2/EXCITED_prepped_data/CT2022.flux1x1-monthly.nc")
ds_cb = excited_workflow.utils.convert_timestamps(ds_cb)

<xarray.Dataset>
Dimensions:          (longitude: 360, latitude: 180, time: 252, components: 6)
Coordinates:
  * longitude        (longitude) float64 -179.5 -178.5 -177.5 ... 178.5 179.5
  * latitude         (latitude) float64 -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * time             (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2020-12-01
Dimensions without coordinates: components
Data variables:
    decimal_time     (time) float64 ...
    time_components  (time, components) int32 ...
    bio_flux_opt     (time, latitude, longitude) float64 ...
    ocn_flux_opt     (time, latitude, longitude) float64 ...
    fossil_flux_imp  (time, latitude, longitude) float64 ...
    fire_flux_imp    (time, latitude, longitude) float64 ...
Attributes:
    averaging_period_length_hours:  744
    email:                          carbontracker.team@noaa.gov
    url:                            http://carbontracker.noaa.gov
    institution:                    NOAA Earth System Research Laboratory
    Conventions:                    CF-1.5
    history:                        Time-stamp: <Orion-login-1.HPC.MsState.Ed...
    NCO:                            netCDF Operators version 4.9.3 (Homepage ...
    version:                        CT2022 1x1 3-hourly fluxes as of 2023-01-...

In [4]:
from xarray_regrid import Grid, create_regridding_dataset

new_grid = Grid(
    north=90,
    east=90,
    south=0,
    west=0,
    resolution_lat=1,
    resolution_lon=1,
)
target_dataset = create_regridding_dataset(new_grid)
target_dataset.latitude.size

91

In [5]:
from excited_workflow.source_datasets import datasets

In [6]:
ds_input = datasets["copernicus_landcover"].load(freq="monthly", target_grid=ds_cb)

In [7]:
ds_input

<xarray.Dataset>
Dimensions:     (latitude: 180, longitude: 360, time: 337)
Coordinates:
  * latitude    (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * longitude   (longitude) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time        (time) datetime64[ns] 1992-01-01 1992-02-01 ... 2020-01-01
Data variables:
    lccs_class  (time, latitude, longitude) float32 dask.array<chunksize=(337, 180, 360), meta=np.ndarray>
Attributes: (12/38)
    id:                         ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992-v2.0.7cds
    title:                      Land Cover Map of ESA CCI brokered by CDS
    summary:                    This dataset characterizes the land cover of ...
    type:                       ESACCI-LC-L4-LCCS-Map-300m-P1Y
    project:                    Climate Change Initiative - European Space Ag...
    references:                 http://www.esa-landcover-cci.org/
    ...                         ...
    geospatial_lon_max:         180
    spatial_resolution:         300m
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.002778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.002778

In [8]:
ds_test = datasets["biomass"].load(freq="monthly", target_grid=ds_cb)

In [9]:
ds_test

<xarray.Dataset>
Dimensions:    (time: 325, latitude: 180, longitude: 360)
Coordinates:
  * time       (time) datetime64[ns] 1995-01-01 1995-02-01 ... 2022-01-01
  * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * longitude  (longitude) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
Data variables:
    biomass    (time, latitude, longitude) float64 dask.array<chunksize=(325, 180, 360), meta=np.ndarray>
Attributes:
    title:        Changes in Global Terrestrial Live Biomass over the 21st Ce...
    version:      2
    institution:  Jet Propulsion Laboratory, California Institute of Technology
    source:       Estimates of carbon stock changes of live woody biomass fro...
    history:      \n2022-08-18: downloaded source from https://zenodo.org/rec...
    references:   \n@Xu2021{\n  author = {Xu, Liang, Saatchi, Sassan S., Yang...